In [37]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('english'))


def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(str.maketrans('', '', string.punctuation))
    filter_str = final_str.translate(str.maketrans('', '', digits))
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

df_test = pd.read_csv('test_dataset.csv')
df_test.head()
df_train = pd.read_csv('train_dataset.csv')

df_train['source_'] = 1
df_test['source_'] =0

df = pd.concat([df_train,df_test])
df['score_1'].fillna(1, inplace = True)
df['score_2'].fillna(1, inplace = True)
df['score_3'].fillna(1, inplace = True)
df['score_4'].fillna(1, inplace = True)
df['score_5'].fillna(1, inplace = True)

df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)

multi_datasets = {}
groups = df.groupby('Essayset')
for name, group in groups:
    multi_datasets[name] = group

pp = []
def different_models(multi_datasets):
    flag = 0
    for i in multi_datasets:
        print(i)
        single_df = multi_datasets[i]
        single_df.reset_index(inplace= True)
        single_df.dropna(axis=0,inplace=True)

        test_id = list(single_df.loc[single_df['source_'] ==0]['ID'])
        es = list(single_df.loc[single_df['source_'] ==0]['Essayset'])
        if len(test_id) != len(es):
            print('PANGA')
            break

        single_df['candi_score'] = single_df[['score_1','score_2' ,'score_3' ,'score_4' ,'score_5']].mean(axis=1)
        single_df['candi_score'] = list(map(lambda x : round(x),single_df['candi_score']))
        single_df.drop(['score_1','score_2' ,'score_3' ,'score_4' ,'score_5','ID','index'],inplace=True,axis=1)

        single_df['kitna_aacha']=single_df['candi_score']

        single_df.drop(['min_score','max_score','Essayset','candi_score'],inplace=True,axis=1)

        all_text  = []
        single_df['EssayText']=single_df['EssayText'].apply(preprocess_data)
        all_text = list(single_df['EssayText'])


        from sklearn.feature_extraction.text import TfidfVectorizer
        tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=20, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
        tfidf.fit(all_text)

        features = tfidf.transform(single_df['EssayText'])

        new_tfidf_features = pd.DataFrame(features.toarray())
        new_tfidf_features['clarity'] = list(single_df['clarity'])
        new_tfidf_features['coherent'] = list(single_df['coherent'])
        new_tfidf_features['kitna_aacha'] = list(single_df['kitna_aacha'])
        new_tfidf_features['source_'] = list(single_df['source_'])

        new_tfidf_features.dropna(axis=0,inplace=True)

        all_preprocessed_single_q_data =  pd.get_dummies(new_tfidf_features,columns=['clarity' ,'coherent'])
        preprocessed_main_test = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 0]
        preprocessed_main_train = all_preprocessed_single_q_data.loc[all_preprocessed_single_q_data['source_'] == 1]

        preprocessed_main_test.drop(['source_','kitna_aacha'],axis =1,inplace=True)

        y_train = (preprocessed_main_train['kitna_aacha'])
        x_train = preprocessed_main_train.drop(['kitna_aacha','source_'],axis =1)

        from sklearn.model_selection import train_test_split
        from sklearn.naive_bayes import MultinomialNB,GaussianNB
        from sklearn.linear_model import LogisticRegression
        X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.2)


        # Import the model we are using
        from sklearn.ensemble import RandomForestClassifier

        # Instantiate model with 1000 decision trees
        rf = RandomForestClassifier()

        # Train the model on training data
        rm = rf.fit(X_train, y_train_internal)
        pred_internal = rm.predict(preprocessed_main_test)
        #pred_internal = rm.predict(preprocessed_main_test)
        res =list(zip(test_id,es,pred_internal))
        #from sklearn import metrics
        if flag ==0 :
            pred_internal = rm.predict(X_test)
            from sklearn import metrics
            pp.append(metrics.accuracy_score(y_test_internal, pred_internal))
        if flag ==1:
            pred_internal = rm.predict(preprocessed_main_test)
            res =list(zip(test_id,es,pred_internal))
            pp.append(res)
    return pp
    #res =list(zip(test_id,pred_internal))



/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [38]:
a =different_models(multi_datasets)

1.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


2.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


3.0


...

/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


9.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


10.0


/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/prashant/miniconda2/envs/p3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [39]:
np.array(a).mean()

0.7518192494432151

In [10]:
from itertools import chain
import itertools
l = list(itertools.chain(*a))


In [14]:
res_ = pd.DataFrame(l,columns=['id','essay_set','essay_score'])


In [16]:
res_.to_csv('baseline_1.csv',index=False)

In [13]:
df_test.head()

,ID,Essayset,min_score,max_score,clarity,coherent,EssayText,source_
0,1673,1,0,3,average,worst,The procedures I think they should have includ...,0
1,1674,1,0,3,average,worst,"In order to replicate this experiment, you wou...",0
2,1675,1,0,3,above_average,above_average,"In order to replicate their experiment, you wo...",0
3,1676,1,0,3,worst,worst,Pleace a simple of one material into one conta...,0
4,1677,1,0,3,worst,worst,Determin the mass of four different samples ma...,0
